# Introduction

This notebook focuses on the application of the [Variational Quantum Eigensolver](https://learning.quantum.ibm.com/tutorial/variational-quantum-eigensolver) (VQE) algorithm to compute the Ground State energy of a physical system described by the Transvrese-field Ising model.

The transverse-field Ising model is a mathematical model in statistical mechanics that features a lattice with nearest neighbour interactions determined by the alignment or anti-alignment of spin projections along the $z$ axis, as well as an external magnetic field perpendicular to the $z$ axis (without loss of generality, along the $x$ axis) which creates an energetic bias for one x-axis spin direction over the other. 

It was first proposed by physicist Ernst Ising in 1925 and has become a fundamental model for studying phase transitions and critical phenomena, especially in the context of magnetism.

### Install dependencies

Note: this notebook is compatible with:

- qiskit                    2.0.0
- qiskit-aer                0.17.0
- qiskit-algorithms         0.3.1
- qiskit-ibm-provider       0.11.0
- qiskit-ibm-runtime        0.37.0
- qiskit-machine-learning   0.8.2
- qiskit-optimization       0.6.1

In [ ]:
# Install packages in the current Jupyter kernel

import sys
!{sys.executable} -m pip install qiskit==2.0.0
!{sys.executable} -m pip install qiskit-aer==0.17.0
!{sys.executable} -m pip install qiskit-algorithms==0.3.1
!{sys.executable} -m pip install qiskit-ibm-provider==0.11.0
!{sys.executable} -m pip install qiskit-ibm-runtime==0.37.0
!{sys.executable} -m pip install qiskit-machine-learning==0.8.2
!{sys.executable} -m pip install qiskit-optimization==0.6.1

In [ ]:
# Basic imports (all others will be imported when needed for teaching purposes)

import numpy as np
import matplotlib.pyplot as plt

import qiskit.version
print("You are using Qiskit "+qiskit.version.get_version_info())


### Connect and configure quantum services

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Select the platform: IBM Quantum (quantum.ibm.com) or IBM Cloud (quantum.cloud.ibm.com)
channel = 'ibm_cloud' #ibm_cloud, #ibn_quantum

if channel == 'ibm_quantum':
  service = QiskitRuntimeService(
    channel = channel,
    # IBM Quantum token
    token = '',
  )

elif channel == 'ibm_cloud':
  service = QiskitRuntimeService(
    channel = channel,
    # IBM Cloud API key
    token = '',
    # IBM Cloud CRN
    instance = ''
  )

In [ ]:
# List all backend names
backends = service.backends()
print([backend.name for backend in backends])

In [ ]:
from qiskit_aer import AerSimulator

# Run type configuration
run_target = 'simulator' #simulator, #least_busy, #any hw printed above

if run_target == 'simulator':
    backend = AerSimulator()

elif run_target == 'least_busy':
    backend = service.least_busy(operational=True, simulator=False)

else:
    backend = service.backend('ibm_sherbrooke')

print("Selected channel: "+channel)
print("Selected backend: "+backend.name)

if ((channel == 'ibm_quantum') and (run_target != 'simulator')):
    print ("-- WARNING: This run may consume several minutes of your Open plan")
elif ((channel == 'ibm_cloud') and (run_target != 'simulator')):
    print ("-- WARNING: This run may consume several credits of your Paygo plan: ~1.6$/second")
elif (run_target == 'simulator'):
    print ("-- This run will be a quick simulation using Qiskit Aer")


### Problem definition

In the Transverse-field Ising model, each site (or lattice point) can have a spin that can be in one of two states: either +1 (spin-up) or -1 (spin-down). Spins are typically arranged on a regular lattice, such as a 1D chain, 2D square grid, or 3D cubic lattice.

The energy of the system is described by the Hamiltonian, which accounts for the interaction between neighboring spins and an external magnetic field. The Hamiltonian is given by:

$H = -J \sum_{<i,j>}{S_i S_j} + b\sum_i S_i$


Where:

- $J$ is the interaction strength between neighboring spins. $J > 0$ corresponds to a ferromagnetic interaction, while $J < 0$ corresponds to an antiferromagnetic interaction
- $<i,j>$ are the nearest neighbors
- $S_i$ is the spin at site $i$
- $b$ is the relative strength of the external field compared to the nearest neighbour interaction of the spins


For a simple 2-spins system, the Hamiltonian simplifies to:

$H = -J S_1 S_2 + b(S_1 + S_2)$

In this notebook, the external magnetic field is considered to be transverse (e.g. on the X-axis).

In [ ]:
from qiskit.quantum_info import SparsePauliOp

# Set up the different observables for the Ising Hamiltonian
observables_labels = ["ZZ", "IX", "XI"]
observables = [SparsePauliOp(label) for label in observables_labels]

# Set up Hamiltonian parameters
J = 1
num_qubits = 2
b_list = np.linspace(0, 4, 10)

def Hamiltonian(J, b, observables):
    H = -J*(observables[0]) + b*(observables[1] + observables[2])
    return H

### Classical solution for benchmarking

In [ ]:
from numpy import linalg

E_l = []
P_l = []
energy_levels = []

for b in range(len(b_list)):

    # Configure the Hamiltonian with Qiskit SparsePauliOp
    H = Hamiltonian(J, b, observables)
    
    # Extract and order eigenvalues (energy)
    E_l, P_l = linalg.eig(H.to_matrix())
    Es = np.sort(E_l)

    energy_levels.append(np.real(Es))

### Quantum approach

##### Ansatz creation
In this example several ansatz creation techniques are included to understand the differences of these approaches.

##### Ansatz transpilation 
To reduce the total job execution time, Qiskit primitives only accept circuits (ansatz) and observables (Hamiltonian) that conform to the instructions and connectivity supported by the target QPU (referred to as instruction set architecture (ISA) circuits and observables).

Schedule a series of qiskit.transpiler  passes to optimize the circuit for a selected backend and make it compatible with the backend's ISA. This can be easily done with a preset pass manager from qiskit.transpiler and its optimization_level parameter.

In [ ]:
from qiskit.circuit.library import efficient_su2
from qiskit.circuit.library import TwoLocal
from qiskit.circuit.library import RealAmplitudes

# Ansatz creation
ans = "efficient"

if ans == "efficient":
    ansatz = efficient_su2(num_qubits, reps=1)
elif ans == "twolocal":
    ansatz = TwoLocal(num_qubits, rotation_blocks=['ry', 'rz'], entanglement_blocks='cx', entanglement='linear', reps=1)
elif ans == "realamplitudes":
    ansatz = RealAmplitudes(num_qubits, entanglement='linear', reps=1)

# Ansatz transpilation
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit import transpile

if(run_target == 'simulator'):
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
    isa_ansatz = pm.run(ansatz)
else:
    isa_ansatz = transpile(ansatz, backend=backend)

# Ansatz drawing
isa_ansatz.decompose().draw("mpl")

##### Cost function creation

The cost function returns the estimation of energy performed by the Estimator primitive

In [ ]:
# Create cost dict to show optimisation progress
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
   }

# Create cost function to be minimized
def cost_fn(params, ansatz, hamiltonian, estimator):

    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    # Packaging all needed variables
    pub = (ansatz, [hamiltonian], [params])
    partial_result = estimator.run(pubs=[pub]).result()
    energy = partial_result[0].data.evs[0]

    # Update dict to show optimisation progress
    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

##### Data vatiables reset

Warning: use only if needed, it will delete all results.

In case of quantum hardware runs, results can be found on quantum.ibm.com or quantum.cloud.ibm.com (depending on the platform chosen).

In [ ]:
# WARNING - Simulated Ground State energy reset
ground_state_energies_simulator = []

In [ ]:
# WARNING - Hardware computed Ground State energy reset
ground_state_energies_hardware = []

##### Quantum runs

In [ ]:
from scipy.optimize import minimize
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator

# Session open
with Session(backend=backend) as session:

    # Initialize Estimator
    estimator = Estimator()

    # Set shot count and options (optional)
    estimator.options.default_shots = 8192
    estimator.options.resilience_level = 2 # Max resilience_level for Estimator is 2. Useful only for noisy simulations or real hardware runs

    # Loop for each field value
    for b in range(len(b_list)):
        
        # 2 qubits Ising Hamiltonian
        H = Hamiltonian(J, b, observables)
        H_isa = H.apply_layout(isa_ansatz.layout)
        
        # Random initial parameters
        initial_point = np.random.rand(ansatz.num_parameters) * 2 * np.pi

        # Optimization process
        res = minimize(
            cost_fn,
            initial_point,
            args=(isa_ansatz, H_isa, estimator),
            method="COBYLA",
            tol=1e-7,
            options={'maxiter': 100}
        )

        # Results extraction
        if run_target == 'simulator':
            ground_state_energies_simulator.append(res.fun)
        else:
            ground_state_energies_hardware.append(res.fun)


#### Plot results

In [ ]:
fig, ax = plt.subplots()
ax.set(xlabel='B field', ylabel='Energy', title='2 qubits Ising Hamiltonian Ground State')

# Classically computed energy levels
ax.plot(b_list, energy_levels, color="#c2c2c2", linewidth=0.5)

# Quantum computed Ground State
ax.scatter(b_list, ground_state_energies_simulator, marker='o', color='b', label="Ground State energy - aer_simulator")

if len(ground_state_energies_hardware) > 0:
    ax.scatter(b_list, ground_state_energies_hardware, marker='^', color='g', label="Ground State energy - "+backend.name)

ax.legend()
plt.show()

In [ ]:
# Print optimization process details

all(cost_history_dict["prev_vector"] == res.x)

fig, ax = plt.subplots()
ax.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"])
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")
plt.draw()